In [1]:
import pandas as pd
import numpy as np

import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from gensim.models import Word2Vec

import spacy
import pandas as pd
import re

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

import pandas as pd
import spacy
from itertools import chain

## Find the Important Features to Predict `Nutrition Density`

In [23]:
# Load dataset
df = pd.read_csv("clean_data_with_same_units.csv", index_col=0)

# Separate predictors and target
X = df.drop(columns=['Nutrition Density', 'food', 'Caloric Value'])
y = df['Nutrition Density']

In [25]:
# Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features (only on training data)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit ElasticNet only on training data
best_en = ElasticNet(alpha=0.01, l1_ratio=0.9, max_iter=5000)
best_en.fit(X_train_scaled, y_train)

# Select important features based on training data
selected_features = X_train.columns[best_en.coef_ != 0]

# Filter dataset using selected features
X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

# Define best hyperparameters manually
best_rf = RandomForestRegressor(
    max_depth=13,
    min_samples_leaf=5,
    min_samples_split=10,
    n_estimators=200,
    random_state=42
)

# Train the Random Forest model with the best hyperparameters
best_rf.fit(X_train_selected, y_train)

# Evaluate on test set
y_pred = best_rf.predict(X_test_selected)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Adjusted R-squared calculation
n = X_test_selected.shape[0]  # number of samples
p = X_test_selected.shape[1]  # number of predictors
adj_r2 = 1 - ((1 - r2) * (n - 1) / (n - p - 1))

# Corrected AIC calculation
rss = mse * n  # Residual Sum of Squares
aic = n * np.log(rss / n) + 2 * (p + 1)

results = {
    "Test MSE": mse,
    "Test RMSE": rmse,
    "Test R2": r2,
    "Test Adjusted R2": adj_r2,
    "Test AIC": aic
}

In [26]:
### **Training Set Evaluation**
y_train_pred = best_rf.predict(X_train_selected)
mse_train = mean_squared_error(y_train, y_train_pred)
rmse_train = np.sqrt(mse_train)
r2_train = r2_score(y_train, y_train_pred)

# Adjusted R-squared for training set
n_train = X_train_selected.shape[0]  # number of training samples
p_train = X_train_selected.shape[1]  # number of predictors
adj_r2_train = 1 - ((1 - r2_train) * (n_train - 1) / (n_train - p_train - 1))

# AIC for training set
aic_train = n_train * np.log(mse_train) + 2 * (p_train + 1)

# Compute Cross-Validation Error
cv_mse = -cross_val_score(best_rf, X_train_selected, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(cv_mse)
cv_r2 = cross_val_score(best_rf, X_train_selected, y_train, cv=5, scoring='r2')

# Adjusted R2 for cross-validation
adj_r2_cv = 1 - ((1 - np.mean(cv_r2)) * (n_train - 1) / (n_train - p_train - 1))

# AIC for cross-validation
aic_cv = n_train * np.log(np.mean(cv_mse)) + 2 * (p_train + 1)

train_results = {
    "Train MSE": mse_train,
    "Train RMSE": rmse_train,
    "Train R2": r2_train,
    "Train Adjusted R2": adj_r2_train,
    "Train AIC": aic_train,
    "Cross-Validation MSE": np.mean(cv_mse),
    "Cross-Validation RMSE": np.mean(cv_rmse),
    "Cross-Validation R2": np.mean(cv_r2),
    "Cross-Validation Adjusted R2": adj_r2_cv,
    "Cross-Validation AIC": aic_cv
}

In [27]:
# Feature Importance
feature_importances = best_rf.feature_importances_
importance_df = pd.DataFrame({'Feature': selected_features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
nutrition_columns = selected_features.tolist()

In [29]:
nutrition_columns

['Fat',
 'Saturated Fats',
 'Monounsaturated Fats',
 'Polyunsaturated Fats',
 'Carbohydrates',
 'Sugars',
 'Protein',
 'Dietary Fiber',
 'Cholesterol',
 'Sodium',
 'Water',
 'Vitamin A',
 'Vitamin B3',
 'Vitamin B5',
 'Vitamin C',
 'Vitamin D',
 'Calcium',
 'Iron',
 'Manganese',
 'Phosphorus',
 'Potassium',
 'Selenium',
 'Zinc']

In [33]:
import json
# Save this list for later use 
with open("nutrition_columns.json", "w") as f:
    json.dump(nutrition_columns, f)


# Save ElasticNet model for feature selection
joblib.dump(best_en, "elastic_net_model.pkl")

# Save RandomForest model for final predictions
joblib.dump(best_rf, "random_forest_model.pkl")

['random_forest_model.pkl']

## NLP - Extract Food Name Components

In [23]:
# Load dataset
path = "clean_data_with_same_units.csv"
df = pd.read_csv(path, index_col=0)

# Store a copy of the original numeric nutrition data
df_original = df.copy()

# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

def parse_food_name(food_name):
    """Extract meaningful components from food name using NLP."""
    doc = nlp(food_name.lower())  # Convert to lowercase
    return [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]  
    # Lemmatization, remove stop-words & non-alphabetic tokens

# Apply parsing
df["parsed_food"] = df["food"].apply(parse_food_name)

# Extract unique components using itertools.chain for better performance
all_components = list(set(chain.from_iterable(df["parsed_food"])))

In [29]:
import json

with open("food_name_components.json", "w") as f:
    json.dump(all_components, f)

## Create a Binary Matrix for Food Components - Using least squares regression to estimate the contribution of each food component to different nutrients

In [43]:
import pandas as pd
import numpy as np

# Create an empty DataFrame for one-hot encoding (food components)
food_component_df = pd.DataFrame(0, index=df.index, columns=all_components, dtype=int)

# Populate the binary matrix using parsed food names
for i, components in enumerate(df["parsed_food"]):
    for component in set(components):  # Avoid duplicate entries
        if component in food_component_df.columns:
            food_component_df.iat[i, food_component_df.columns.get_loc(component)] = 1

# Merge one-hot encoded matrix back with the original DataFrame
df = pd.concat([df, food_component_df], axis=1)

# Get numeric nutrition columns from the original dataset (excluding target variable)
nutrition_columns = df_original.select_dtypes(include=[np.number]).columns.drop("Nutrition Density")

# Convert numeric columns to float and fill NaNs with 0
df_original[nutrition_columns] = df_original[nutrition_columns].apply(pd.to_numeric, errors="coerce").fillna(0)

# Prepare matrices
X = food_component_df.values  # One-hot encoded matrix (food × components)
Y = df_original[nutrition_columns].values.astype(np.float64)  # Nutrient matrix (food × nutrients)

# Solve least squares problem to estimate nutrient contributions
W, residuals, rank, s = np.linalg.lstsq(X, Y, rcond=None)

# Convert results into a DataFrame (components × nutrients)
component_nutrition_contributions = pd.DataFrame(W, index=food_component_df.columns, columns=nutrition_columns)

In [48]:
# Save results for later use in the app
component_nutrition_contributions.to_csv("component_nutrition_contributions.csv")
component_nutrition_contributions.to_json("component_nutrition_contributions.json")

## NLP - Use word embeddings - Match new/unknown food names to the most similar ones in existing food dictionary

In [55]:
import spacy
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load medium-sized spaCy model (better than en_core_web_sm)
nlp = spacy.load("en_core_web_md")

# Load food components from pickle
with open("food_components.pkl", "rb") as f:
    all_components = pickle.load(f)

# Load nutrition data from CSV
component_nutrition_contributions = pd.read_csv("component_nutrition_contributions.csv", index_col=0)

# Store precomputed word vectors for all known components (avoid recomputation)
component_vectors = {}
for word in all_components:
    word_vector = nlp(word).vector
    if np.any(word_vector):  # Ensure vector exists (some words may be OOV)
        component_vectors[word] = word_vector

# Save precomputed vectors for fast future use
with open("component_vectors.pkl", "wb") as f:
    pickle.dump(component_vectors, f)


def find_closest_component(user_input):
    """Find the closest matching food component using NLP embeddings."""
    user_words = parse_food_name(user_input)  # Extract words from input
    matched_words = []

    for word in user_words:
        # **Step 1: Check for an exact match**
        if word in all_components:
            matched_words.append(word)
            continue  # Skip similarity check if exact match found

        # **Step 2: Compute similarity with known components**
        word_vector = nlp(word).vector
        if not np.any(word_vector):  # Skip words without a vector
            continue

        word_vector = word_vector.reshape(1, -1)

        similarities = {
            comp: cosine_similarity(word_vector, comp_vec.reshape(1, -1))[0][0]
            for comp, comp_vec in component_vectors.items()
        }

        # Get best match
        best_match = max(similarities, key=similarities.get)
        best_match_score = similarities[best_match]

        # **Avoid poor matches** (threshold > 0.5)
        if best_match_score > 0.5:
            matched_words.append(best_match)

    return matched_words


In [59]:
# Example usage
user_input = "bbq rice flour"
matched_components = find_closest_component(user_input)
print(f"Matched Components: {matched_components}")

Matched Components: ['pork', 'rice', 'flour']


### Save the model

In [61]:
script = """import spacy
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Load medium-sized spaCy model (better than en_core_web_sm)
nlp = spacy.load("en_core_web_md")

# Load food components from pickle
with open("food_components.pkl", "rb") as f:
    all_components = pickle.load(f)

# Load nutrition data
component_nutrition_contributions = pd.read_csv("component_nutrition_contributions.csv", index_col=0)

# Precompute word vectors for all known food components
component_vectors = {}
for word in all_components:
    word_vector = nlp(word).vector
    if np.any(word_vector):  # Ensure word has a vector (avoid OOV issues)
        component_vectors[word] = word_vector

# Save precomputed vectors for fast reuse
with open("component_vectors.pkl", "wb") as f:
    pickle.dump(component_vectors, f)

def parse_food_name(food_name):
    '''Extract meaningful words from food names.'''
    doc = nlp(food_name.lower())
    return [token.lemma_ for token in doc if not token.is_stop and token.is_alpha]

def find_closest_component(user_input):
    '''Find the closest matching food component using NLP embeddings.'''
    user_words = parse_food_name(user_input)  
    matched_words = []

    for word in user_words:
        # Step 1: Check for an exact match
        if word in all_components:
            matched_words.append(word)
            continue  # Skip similarity check if exact match found

        # Step 2: Compute similarity with known components
        word_vector = nlp(word).vector
        if not np.any(word_vector):  # Skip words without a vector
            continue

        word_vector = word_vector.reshape(1, -1)

        similarities = {
            comp: cosine_similarity(word_vector, comp_vec.reshape(1, -1))[0][0]
            for comp, comp_vec in component_vectors.items()
        }

        # Get best match
        best_match = max(similarities, key=similarities.get)
        best_match_score = similarities[best_match]

        # Avoid poor matches (threshold > 0.5)
        if best_match_score > 0.5:
            matched_words.append(best_match)

    return matched_words
"""

# Save script to a file
with open("food_matching.py", "w") as f:
    f.write(script)

print("food_matching.py saved successfully!")


food_matching.py saved successfully!


In [63]:
from food_matching import find_closest_component

user_food = "grilled chicken breast"
matched_components = find_closest_component(user_food)

print(matched_components)

['grill', 'chicken', 'breast']


In [35]:
import json
import joblib
import pandas as pd

# Load trained RandomForest model
best_rf = joblib.load("random_forest_model.pkl")

# Load selected feature names (used during training)
with open("nutrition_columns.json", "r") as f:
    nutrition_columns = json.load(f)

# Load food component nutrient contributions
component_nutrition_contributions = pd.read_csv("component_nutrition_contributions.csv", index_col=0)

# Ensure we only use features present in both `nutrition_columns` and the dataset
nutrition_columns = [col for col in nutrition_columns if col in component_nutrition_contributions.columns]

# Select only the columns that were used in training
X_components = component_nutrition_contributions[nutrition_columns]

# Ensure the feature order matches what was used in training
expected_features = list(best_rf.feature_names_in_)  # Features seen during training

# Find the intersection between expected features and current dataset
matching_features = [col for col in expected_features if col in X_components.columns]

# Reorder X_components to match training order
X_components = X_components[matching_features]

# Check if feature names exactly match before prediction
assert list(X_components.columns) == list(best_rf.feature_names_in_), "Feature names do not match!"

# Predict Nutrition Density
predicted_nutrition_density = best_rf.predict(X_components)

# Add predictions to DataFrame
component_nutrition_contributions["Predicted_Nutrition_Density"] = predicted_nutrition_density

# Save predictions
component_nutrition_contributions.to_csv("component_nutrition_density_predictions.csv")
component_nutrition_contributions.to_json("component_nutrition_density_predictions.json")

print("✅ Predicted Nutrition Density saved successfully!")


✅ Predicted Nutrition Density saved successfully!
